# Generate Sentiment Data

Import libraries

In [29]:
import json
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

c:\users\iceok\documents\.virtualenvs\bsent\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 112 from C header, got 124 from PyObject
  return f(*args, **kwds)
c:\users\iceok\documents\.virtualenvs\bsent\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 112 from C header, got 124 from PyObject
  return f(*args, **kwds)


## Get Data

In [3]:
with open('data/cnn_articles.json', 'r') as f:
    cnn_articles = json.load(f)

In [4]:
with open('data/guardian_articles.json', 'r') as f:
    guardian_articles = json.load(f)

## Sentiment Analysis

In [5]:
analyser = SentimentIntensityAnalyzer()

In [25]:
def generate_sentiment(articles):
    result = []
    for article in articles:
        
        # Generate Polarity Scores
        title_sentiment = analyser.polarity_scores(article['title'])
        full_sentiment = analyser.polarity_scores(article['full_article'])
        
        # Generate row of data
        row = {}
        row['publish_date'] = article['publish_date']
        row['title'] = article['title']
        row['title_sentiment_pos'] = title_sentiment['pos']
        row['title_sentiment_neu'] = title_sentiment['neu']
        row['title_sentiment_neg'] = title_sentiment['neg']
        row['title_sentiment_comp'] = title_sentiment['compound']
        row['full_sentiment_pos'] = full_sentiment['pos']
        row['full_sentiment_neu'] = full_sentiment['neu']
        row['full_sentiment_neg'] = full_sentiment['neg']
        row['full_sentiment_comp'] = full_sentiment['compound']
        
        # Add to results
        result.append(row)
        
    return result

In [26]:
cnn_sentiment = generate_sentiment(cnn_articles)

In [27]:
guardian_sentiment = generate_sentiment(guardian_articles)

## Load in a dataframe and pickle

In [34]:
cnn_df = pd.DataFrame(cnn_sentiment)
cnn_df['source'] = 'CNN'

In [35]:
guardian_df = pd.DataFrame(guardian_sentiment)
guardian_df['source'] = 'Guardian'

In [44]:
sentiment_df = pd.concat([cnn_df, guardian_df]).reset_index(drop=True)
sentiment_df['publish_date'] = pd.to_datetime(sentiment_df.publish_date)

In [46]:
sentiment_df.head()

,publish_date,title,title_sentiment_pos,title_sentiment_neu,title_sentiment_neg,title_sentiment_comp,full_sentiment_pos,full_sentiment_neu,full_sentiment_neg,full_sentiment_comp,source
0,2019-08-30 00:00:00,It'll take superpowers to unseat Boris Johnson...,0.141,0.859,0.000,0.3182,0.089,0.862,0.050,0.9909,CNN
1,2019-08-30 00:00:00,Portuguese woman interrupts Sky broadcast with...,0.000,0.753,0.247,-0.3182,0.053,0.899,0.048,0.5034,CNN
2,2019-08-29 00:00:00,Boris Johnson put the Queen in an absurdly awk...,0.000,0.849,0.151,-0.1531,0.050,0.907,0.044,0.4576,CNN
3,2019-08-29 00:00:00,"5 things to know for August 29: Dorian, citize...",0.000,1.000,0.000,0.0000,0.054,0.869,0.077,-0.9614,CNN
4,2019-08-28 09:36:30+00:00,Boris Johnson asks Queen to suspend UK Parliam...,0.000,0.813,0.187,-0.3182,0.024,0.832,0.143,-0.9892,CNN


In [47]:
sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36343 entries, 0 to 36342
Data columns (total 11 columns):
publish_date            36343 non-null object
title                   36343 non-null object
title_sentiment_pos     36343 non-null float64
title_sentiment_neu     36343 non-null float64
title_sentiment_neg     36343 non-null float64
title_sentiment_comp    36343 non-null float64
full_sentiment_pos      36343 non-null float64
full_sentiment_neu      36343 non-null float64
full_sentiment_neg      36343 non-null float64
full_sentiment_comp     36343 non-null float64
source                  36343 non-null object
dtypes: float64(8), object(3)
memory usage: 2.6+ MB


On first glance, we can tell the following from our data:
* Both the title and full article sentiment are primarily neutral (~80%)
* The sentiment of the title has more variance than that of the full article however its the opposite for the compound variance.

In [48]:
sentiment_df.describe()

,title_sentiment_pos,title_sentiment_neu,title_sentiment_neg,title_sentiment_comp,full_sentiment_pos,full_sentiment_neu,full_sentiment_neg,full_sentiment_comp
count,36343.000000,36343.000000,36343.000000,36343.000000,36343.000000,36343.000000,36343.000000,36343.000000
mean,0.079851,0.812731,0.107421,-0.046296,0.097808,0.824778,0.076616,0.363296
std,0.122744,0.168914,0.140056,0.363943,0.032651,0.051967,0.034582,0.844047
min,0.000000,0.185000,0.000000,-0.939300,0.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.693000,0.000000,-0.318200,0.077000,0.796000,0.053000,-0.771700
50%,0.000000,0.809000,0.000000,0.000000,0.097000,0.825000,0.073000,0.949900
75%,0.167000,1.000000,0.208000,0.077200,0.117000,0.855000,0.096000,0.993200
max,0.778000,1.000000,0.732000,0.927400,0.336000,1.000000,0.623000,1.000000


In [49]:
sentiment_df.to_pickle('data/sentiment_df.pickle')

## END